In [1]:
import os
import datetime
import pandas as pd
from polygon import RESTClient
import pytz
eastern = 'US/Eastern'

In [2]:
trade_map={"I":"orig_id","x":"exchange","p":"price","i":"id","e":"correction","r":"trf_id","t":"sip_timestamp","y":"participant_timestamp","f":"trf_timestamp","q":"sequence_number","c":"conditions","s":"size","z":"tape"
}

trade_conditions = {'0': 'Regular', '1': 'Acquisition', '2': 'AveragePrice', '3': 'AutomaticExecution', '4': 'Bunched', '5': 'BunchSold', '6': 'CAPElection', '7': 'CashTrade', '8': 'Closing', '9': 'Cross', '10': 'DerivativelyPriced', '11': 'Distribution', '12': 'FormT(ExtendedHours)', '13': 'FormTOutOfSequence', '14': 'InterMarketSweep', '15': 'MarketCenterOfficialClose', '16': 'MarketCenterOfficialOpen', '17': 'MarketCenterOpening', '18': 'MarketCenterReOpenning', '19': 'MarketCenterClosing', '20': 'NextDay', '21': 'PriceVariation', '22': 'PriorReferencePrice', '23': 'Rule155Amex', '24': 'Rule127Nyse', '25': 'Opening', '26': 'Opened', '27': 'RegularStoppedStock', '28': 'ReOpening', '29': 'Seller', '30': 'SoldLast', '31': 'SoldLastStoppedStock', '32': 'SoldOutOfSequence', '33': 'SoldOutOfSequenceStoppedStock', '34': 'Split', '35': 'StockOption', '36': 'YellowFlag', '37': 'OddLot', '38': 'CorrectedConsolidatedClosePrice', '39': 'Unknown', '40': 'Held', '41': 'TradeThruExempt', '42': 'NonEligible', '43': 'NonEligible-extended', '44': 'Cancelled', '45': 'Recovery', '46': 'Correction', '47': 'AsOf', '48': 'AsOfCorrection', '49': 'AsOfCancel', '50': 'OOB', '51': 'Summary', '52': 'Contingent', '53': 'Contingent(Qualified)', '54': 'Errored'}

In [3]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"

def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_list_of_Exchanges():

    with RESTClient(key) as client:
        respex = client.stocks_equities_exchanges()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict

def get_list_of_trade_conditions():

    with RESTClient(key) as client:
        respex = client.stocks_equities_condition_mappings()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict
    
def update_condition_labels(keyFunction, values):
    k =  list(keyFunction[str(v)] for v in list(values))
    return ", ".join(k)


In [4]:
exchange_list_dict= get_list_of_Exchanges()
trade_conds = pd.DataFrame(list(trade_conditions.items()),columns = ['condition','meaning']) 
trade_conds.to_csv('trade_cond.csv')

In [5]:
def get_trades(ticker, date, size):
    
    with RESTClient(key) as client:
        resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, limit=size, reverse=False)
        return resp.__dict__
    
def get_trades_all(ticker, date, size):
    total_data = []
    lasttimestamp = 0
    while True:
        with RESTClient(key) as client:
            if lasttimestamp == 0:
                resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, limit=size, reverse=False)
            else:
                resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, timestamp = lasttimestamp, limit=size, reverse=False)
            result = resp.__dict__
            if "results" in result:
                result = result['results']
            else:
                break
            lasttimestamp = result[-1]['t']
            total_data.extend(result)
            if len(result) == 50000:
                continue
            else:
                break
    return total_data

In [6]:
def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified

def data_preprocessing(trades):
    trades_df = pd.DataFrame(trades).rename(columns=trade_map)
    trades_df['sip_timestamp'] =pd.to_datetime(trades_df['sip_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)
    trades_df['participant_timestamp'] =pd.to_datetime(trades_df['participant_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)

    trades_df['hours'] = trades_df['sip_timestamp'].apply(lambda x: x.hour)
    trades_df['minutes'] = trades_df['sip_timestamp'].apply(lambda x: x.minute)
    trades_df['seconds'] = trades_df['sip_timestamp'].apply(lambda x: x.second)
    
    trades_df['time']=trades_df['sip_timestamp'].dt.time
    trades_df['conditions_string'] = trades_df['conditions'].astype(str)

    return trades_df

def extract_opening_data(trades_df):
    trades_df['conditions_string'] = trades_df['conditions'].astype(str)

    open_= trades_df[trades_df.conditions_string.str.contains("16")]
    open_ = open_.sort_values('size',ascending=False)

    open_price= list(open_['price'])[0]
    open_size= list(open_['size'])[0]
    open_datetime = list(open_['sip_timestamp'])
    open_time = list(open_['sip_timestamp'].dt.time)[0]
    
    to_return = {
        'open_price':open_price,
        'open_time':open_time,
        'open_size':open_size
    }
    
    return to_return

def extract_closing_data(trades_df):
    
    close= trades_df[trades_df.conditions_string.str.contains("15")]
    close = close.sort_values('size',ascending=False)
    close_price= list(close['price'])[0]
    close_size= list(close['size'])[0]
    close_datetime = list(close['sip_timestamp'])[0]
    close_time = list(close['sip_timestamp'].dt.time)[0]
    
    to_return = {
        'close_price':close_price,
        'close_time':close_time,
        'close_size':close_size
    }
    return to_return

def aggregate_premarket_data(trades_df, open_time):
    premarket= trades_df[trades_df.time < open_time]
    pre_max_price = premarket.price.max()
    pre_min_price = premarket.price.min()
    pre_std_price = premarket.price.std()
    pre_median_price = premarket.price.quantile(0.5)
    pre_volume = premarket.size.sum()
    
    to_return = {
        'pre_max_price':pre_max_price,
        'pre_min_price': pre_min_price,
        'pre_std_price': pre_std_price,
        'pre_median_price': pre_median_price,
        'pre_volume':pre_volume
    }
    
    return to_return

def aggregate_postmarket_data(trades_df, close_time):
    postmarket= trades_df[trades_df.time > close_time]
    post_max_price = postmarket.price.max()
    post_min_price = postmarket.price.min()
    post_std_price = postmarket.price.std()
    post_median_price = postmarket.price.quantile(0.5)
    post_volume = postmarket.size.sum()

    
    to_return = {
        'post_max_price':post_max_price,
        'post_min_price':post_min_price, 
        'post_std_price':post_std_price,
        'post_median_price':post_median_price, 
        'post_volume':post_volume
    }
    return to_return

def append_aggregation_to_df():
    return


# MANUAL PROCESS

- make request
- create additional columns
- extract open and close data
- aggregate pre and post market data
- append to df


In [7]:
main_df = pd.read_csv('stocks-profile.csv')


In [8]:
# Extract required symbols from data
q_sym = extract_qualified_symbols(main_df, 4, 2)
q_sym_list = list(q_sym['symbol'])


In [9]:
#Get polygon data for symbol
response_data = get_trades_all(q_sym_list[0], "2021-02-17", 50000)

In [10]:
resp_df = data_preprocessing(response_data)


In [11]:
open_data_extract = extract_opening_data(resp_df)

close_data_extract = extract_closing_data(resp_df)

premarket_aggregate = aggregate_premarket_data(resp_df, open_data_extract['open_time'])

postmarket_aggregate = aggregate_postmarket_data(resp_df, close_data_extract['close_time'])


In [12]:

merged_data = {**open_data_extract, **close_data_extract, **premarket_aggregate, **postmarket_aggregate, **{"symbol":q_sym_list[0]}}
print(merged_data)


{'open_price': 778.17, 'open_time': datetime.time(9, 30, 0, 923676), 'open_size': 282823, 'close_price': 798.15, 'close_time': datetime.time(16, 0, 0, 338520), 'close_size': 284787, 'pre_max_price': 796.6157, 'pre_min_price': 775.89, 'pre_std_price': 4.960197552634571, 'pre_median_price': 781.9, 'pre_volume': 1160913, 'post_max_price': 839.549744, 'post_min_price': 780.8471, 'post_std_price': 1.4050200199153364, 'post_median_price': 795.0, 'post_volume': 207009, 'symbol': 'TSLA'}


In [13]:
pd.DataFrame([merged_data])

,open_price,open_time,open_size,close_price,close_time,close_size,pre_max_price,pre_min_price,pre_std_price,pre_median_price,pre_volume,post_max_price,post_min_price,post_std_price,post_median_price,post_volume,symbol
0,778.17,09:30:00.923676,282823,798.15,16:00:00.338520,284787,796.6157,775.89,4.960198,781.9,1160913,839.549744,780.8471,1.40502,795.0,207009,TSLA


# AUTOMATED CODE VERSION

In [14]:
def batch_symbols_processing(df, pvlvl, advlvl, date):
    q_sym = extract_qualified_symbols(main_df, pvlvl, advlvl)
    q_sym_list = q_sym['symbol']
    final_result = []

    for sym in q_sym_list:
        response_data = get_trades_all(str(sym), date, 50000)
        resp_df = data_preprocessing(response_data)

        open_data_extract = extract_opening_data(resp_df)
        close_data_extract = extract_closing_data(resp_df)

        premarket_aggregate = aggregate_premarket_data(resp_df, open_data_extract['open_time'])
        postmarket_aggregate = aggregate_postmarket_data(resp_df, close_data_extract['close_time'])


        merged_data = {**open_data_extract, **close_data_extract, **premarket_aggregate, **postmarket_aggregate, **{"symbol":sym}}

        print(sym)
        print("done")
        
        final_result.append(merged_data)
    
    return pd.DataFrame(final_result)


In [15]:
complete_result = batch_symbols_processing(main_df, 4, 2, '2021-02-17')

TSLA
done
AMZN
done
DIS
done
MSFT
done
NFLX
done
ZM
done
BA
done
MRNA
done
GOOGL
done
NVDA
done
BYND
done
BABA
done
FB
done
SQ
done
COST
done
JNJ
done
PYPL
done
TGT
done
V
done
MMM
done
ROKU
done
TDOC
done
CRM
done
MCD
done
ABNB
done
NVAX
done
CRWD
done
MA
done
UPS
done
ADBE
done
ENPH
done
GOOG
done
LULU
done
TWLO
done
CAT
done
LMT
done
CVNA
done
OKTA
done
UNH
done
DOCU
done
SNOW
done
ETSY
done
BIDU
done
HD
done
DASH
done
APPN
done
CRSP
done
FDX
done
AI
done
SE
done
LIN
done
LOW
done
W
done
CLX
done
FLGT
done
SEDG
done
SWKS
done
GS
done
Z
done
ZS
done
LLY
done
AVGO
done
HON
done
PDD
done
SPLK
done
CCI
done
MTCH
done
DG
done
TEAM
done
ALB
done
VRTX
done
TMO
done
AMT
done
GD
done
PNC
done
BIIB
done
WDAY
done
TXN
done
DHR
done
NXPI
done
CI
done
UNP
done
AMGN
done
DE
done
ALXN
done
ACN
done
HCA
done
ADP
done
MCHP
done
ADI
done
CME
done
ZTS
done
GPN
done
CB
done
SPGI
done
AON
done


In [16]:
complete_result.head()

,open_price,open_time,open_size,close_price,close_time,close_size,pre_max_price,pre_min_price,pre_std_price,pre_median_price,pre_volume,post_max_price,post_min_price,post_std_price,post_median_price,post_volume,symbol
0,778.17,09:30:00.923676,282823,798.15,16:00:00.338520,284787,796.6157,775.89,4.960198,781.90,1160913,839.549744,780.8471,1.405020,795.00,207009,TSLA
1,3262.15,09:30:01.830429,42452,3308.64,16:00:00.479632,135221,3279.9500,3252.00,7.250585,3265.00,58752,3315.000000,3285.0000,1.604712,3308.64,39610,AMZN
2,185.57,09:31:13.059894,252300,186.44,16:04:29.488210,441180,186.4800,185.00,0.394767,185.43,46342,186.990000,184.9743,0.144455,186.49,25381,DIS
3,241.00,09:30:02.769869,547102,244.20,16:00:00.834079,2936573,243.7000,240.60,0.737206,241.98,76330,251.322000,243.9000,0.194155,244.15,26418,MSFT
4,551.00,09:30:01.695822,56286,551.34,16:00:00.457967,141609,559.5000,546.89,3.056515,553.88,20832,556.000000,548.0100,0.709501,551.34,8512,NFLX


In [18]:
complete_result.to_csv("data_pv4_adv2.csv")